Read in a single file and make duplicates

In [1]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from delta import *
from delta.tables import DeltaTable
import json
import pandas as pd
import datetime
import time

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000) # Adjust as needed
pd.set_option('display.max_colwidth', None)

spark = (pyspark.sql.SparkSession.builder.appName("MyApp").master("local[*]")
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.1.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore")
.getOrCreate())
spark.sparkContext.setLogLevel('ERROR')

:: loading settings :: url = jar:file:/home/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/sparkuser/.ivy2/cache
The jars for the packages stored in: /home/sparkuser/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ca5819cc-43ab-4942-8efb-c782527bd356;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-spark_2.12/3.1.0/delta-spark_2.12-3.1.0.jar ...
	[SUCCESSFUL ] io.delta#delta-spark_2.12;3.1.0!delta-spark_2.12.jar (2890ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/3.1.0/delta-storage-3.1.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;3.1.0!delta-storage.jar (33ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.9.3/antlr4-runtime-4.9.3.jar ...
	[SUCCESSFUL ] org.antlr#antlr4-runtime;4.9.3!antlr4-runtime.jar (85ms)
:: resolution report :: resolve 5662

In [2]:
!rm -r spark

rm: cannot remove 'spark': No such file or directory


In [4]:
spark.sql("DROP TABLE IF EXISTS spark")

DataFrame[]

make several iterations of changes

In [5]:
df_array = ["""{"id": "2111", "name": "OLIVIA", "age": "37", "metadata":[["a",1], ["b",2]]}"""]
df_array = spark.sparkContext.parallelize(df_array)
df_array = spark.read.json(df_array)
df_array.show()

+---+----+----------------+------+
|age|  id|        metadata|  name|
+---+----+----------------+------+
| 37|2111|[[a, 1], [b, 2]]|OLIVIA|
+---+----+----------------+------+



In [6]:
df_array.write.format("delta").mode("overwrite").save("spark")
time.sleep(3)

In [7]:
df_array_2 = df_array.withColumn("name", F.lower(F.col("name")))

In [8]:
df_array_2.write.format("delta").mode("overwrite").save("spark")
time.sleep(3)

In [9]:
df_array_3 = df_array_2.withColumn("name", F.concat(F.lit("abab "), F.col("name")))

In [10]:
df_array_3.write.format("delta").mode("overwrite").save("spark")
time.sleep(3)

In [11]:
spark.read.format("delta").load("spark").toPandas()

,age,id,metadata,name
0,37,2111,"[[a, 1], [b, 2]]",abab olivia


know how files are layed out in the delta lakes format

In [12]:
!ls spark

_delta_log
part-00000-0aac5455-055c-4fd8-8eff-4b3c478fe554-c000.snappy.parquet
part-00000-253893b7-4e39-491d-a386-0338f7575ce3-c000.snappy.parquet
part-00000-872ffadb-d922-4407-a6e9-c722cc6eab24-c000.snappy.parquet
part-00007-51640031-68de-4db2-a3a4-d16dfe2598b2-c000.snappy.parquet
part-00007-6fcf38e4-466f-48d0-853f-c22a1d3dd9da-c000.snappy.parquet
part-00007-a2d140ac-0137-457e-8cb7-8c96eb1d0b68-c000.snappy.parquet


In [13]:
!ls spark/_delta_log

00000000000000000000.json  00000000000000000001.json  00000000000000000002.json


look at the tables history

In [14]:
delta_table = DeltaTable.forPath(spark, "spark")
history_df = delta_table.history()
history_df.toPandas()

,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,2,2025-02-13 21:21:30.272,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,1.0,Serializable,False,"{'numOutputRows': '1', 'numOutputBytes': '2111', 'numFiles': '2'}",None,Apache-Spark/3.5.4 Delta-Lake/3.1.0
1,1,2025-02-13 21:21:26.373,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,0.0,Serializable,False,"{'numOutputRows': '1', 'numOutputBytes': '2076', 'numFiles': '2'}",None,Apache-Spark/3.5.4 Delta-Lake/3.1.0
2,0,2025-02-13 21:21:21.291,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,NaN,Serializable,False,"{'numOutputRows': '1', 'numOutputBytes': '2076', 'numFiles': '2'}",None,Apache-Spark/3.5.4 Delta-Lake/3.1.0


restoring a table to a previous version based on ID

In [15]:
delta_table.restoreToVersion(1)

DataFrame[table_size_after_restore: bigint, num_of_files_after_restore: bigint, num_removed_files: bigint, num_restored_files: bigint, removed_files_size: bigint, restored_files_size: bigint]

In [16]:
delta_table = DeltaTable.forPath(spark, "spark")
history_df = delta_table.history()
history_df.toPandas()

,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,3,2025-02-13 21:21:45.673,None,None,RESTORE,"{'version': '1', 'timestamp': None}",None,None,None,2.0,Serializable,False,"{'removedFilesSize': '1390', 'tableSizeAfterRestore': '1355', 'numRemovedFiles': '1', 'restoredFilesSize': '1355', 'numOfFilesAfterRestore': '1', 'numRestoredFiles': '1'}",None,Apache-Spark/3.5.4 Delta-Lake/3.1.0
1,2,2025-02-13 21:21:30.272,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,1.0,Serializable,False,"{'numOutputRows': '1', 'numOutputBytes': '2111', 'numFiles': '2'}",None,Apache-Spark/3.5.4 Delta-Lake/3.1.0
2,1,2025-02-13 21:21:26.373,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,0.0,Serializable,False,"{'numOutputRows': '1', 'numOutputBytes': '2076', 'numFiles': '2'}",None,Apache-Spark/3.5.4 Delta-Lake/3.1.0
3,0,2025-02-13 21:21:21.291,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,NaN,Serializable,False,"{'numOutputRows': '1', 'numOutputBytes': '2076', 'numFiles': '2'}",None,Apache-Spark/3.5.4 Delta-Lake/3.1.0


In [17]:
timestamp_we_want_to_rollback_to = delta_table.history().where(F.col("version") == 1).select("timestamp").collect()[0]['timestamp'].strftime("%Y-%m-%d %H:%M:%S")
timestamp_we_want_to_rollback_to

'2025-02-13 21:21:26'

In [18]:
delta_table.restoreToTimestamp(timestamp_we_want_to_rollback_to)

DataFrame[table_size_after_restore: bigint, num_of_files_after_restore: bigint, num_removed_files: bigint, num_restored_files: bigint, removed_files_size: bigint, restored_files_size: bigint]

In [19]:
delta_table = DeltaTable.forPath(spark, "spark")
history_df = delta_table.history()
history_df.toPandas()

,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,4,2025-02-13 21:21:48.427,None,None,RESTORE,"{'version': None, 'timestamp': '2025-02-13 21:21:26.0'}",None,None,None,3.0,Serializable,False,"{'removedFilesSize': '1355', 'tableSizeAfterRestore': '1355', 'numRemovedFiles': '1', 'restoredFilesSize': '1355', 'numOfFilesAfterRestore': '1', 'numRestoredFiles': '1'}",None,Apache-Spark/3.5.4 Delta-Lake/3.1.0
1,3,2025-02-13 21:21:45.673,None,None,RESTORE,"{'version': '1', 'timestamp': None}",None,None,None,2.0,Serializable,False,"{'removedFilesSize': '1390', 'tableSizeAfterRestore': '1355', 'numRemovedFiles': '1', 'restoredFilesSize': '1355', 'numOfFilesAfterRestore': '1', 'numRestoredFiles': '1'}",None,Apache-Spark/3.5.4 Delta-Lake/3.1.0
2,2,2025-02-13 21:21:30.272,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,1.0,Serializable,False,"{'numOutputRows': '1', 'numOutputBytes': '2111', 'numFiles': '2'}",None,Apache-Spark/3.5.4 Delta-Lake/3.1.0
3,1,2025-02-13 21:21:26.373,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,0.0,Serializable,False,"{'numOutputRows': '1', 'numOutputBytes': '2076', 'numFiles': '2'}",None,Apache-Spark/3.5.4 Delta-Lake/3.1.0
4,0,2025-02-13 21:21:21.291,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,NaN,Serializable,False,"{'numOutputRows': '1', 'numOutputBytes': '2076', 'numFiles': '2'}",None,Apache-Spark/3.5.4 Delta-Lake/3.1.0


query the based off of history

In [20]:
spark.read.format("delta").option("timestampAsOf", timestamp_we_want_to_rollback_to).load("spark").toPandas()

,age,id,metadata,name
0,37,2111,"[[a, 1], [b, 2]]",OLIVIA


In [21]:
spark.read.format("delta").option("versionAsOf", 1).load("spark").toPandas()

,age,id,metadata,name
0,37,2111,"[[a, 1], [b, 2]]",olivia
